# Introduction to Automation with LangChain, Generative AI, and Python
**2.8: Conversation Summary Memory**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

We will now look at an entirely different way to maintain chat histories for memory. Rather than simply forgetting the oldest parts of the chat, the ConversationSummaryMemory memory object always maintains a summary of the conversation and then updates that summary as the conversation progresses. This process can help to preserve important information even if it occurred earlier in the conversation.

We first examine ConversationSummaryMemory is a dynamic memory model designed to manage the flow of a dialogue by retaining a summary of the conversation. This method ensures that the memory buffer maintains a manageable size, avoiding overflow and performance issues that can arise with an excessively large interaction history.

The provided code snippet demonstrates how to set up and use a conversational AI system using the LangChain framework, specifically focusing on maintaining a concise memory of recent interactions. The ConversationChain class orchestrates the conversation flow, integrating various components such as language model, memory, and prompt formatting.

Firstly, several modules are imported, including ConversationChain for managing the conversation, ConversationSummaryMemory for handling the memory of recent interactions, and ChatOpenAI to interface with OpenAI's language models. The PromptTemplate is used to define the structure of the prompts sent to the language model.

In the setup, MODEL specifies the particular version of the language model (Claude 3), and TEMPLATE outlines the format of the conversation, integrating previous dialogue history and the current user input into the prompt. This structured prompt is then used to create a PromptTemplate object.

The begin_conversation function initializes the language model and sets up the memory to store the summary. This summary helps keep the conversation relevant and efficient by focusing on the most important aspects of exchanges. The ConversationChain object is created with the prompt template, the language model, and the memory buffer, and it's ready to process conversation inputs.

The converse function takes a conversation object and a user prompt to produce responses. It uses the conversation object's invoke method, which applies the prompt template, consults the memory, and generates a response based on the current and recent dialogue context.

Overall, this setup is optimized for creating a responsive and context-aware conversational AI that doesn't overload with too much past information, maintaining relevancy and efficiency in ongoing interactions.

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain_aws import ChatBedrock
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'
TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def begin_conversation(summary_llm):
    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
    )

    # Initialize memory and conversation
    memory = ConversationSummaryMemory(llm=summary_llm)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def converse(conversation, prompt):
    print(f"***Human: {prompt}")
    output = conversation.invoke(prompt)
    return output['response']

We can now carry on a simple conversation with the LLM, using LangChain to track the conversation memory.

In [2]:
# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.7},
)

conversation = begin_conversation(llm)
output = converse(conversation, "Hello, what is my name?")
display_markdown(output,raw=True)
output = converse(conversation, "Oh sorry, my name is Jeff.")
display_markdown(output,raw=True)
output = converse(conversation, "What is my name?")
display_markdown(output,raw=True)

***Human: Hello, what is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant created by Anthropic, I don't have personal information about you unless you provide it to me.
```

***Human: Oh sorry, my name is Jeff.


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I don't have a personal identity, but I'm here to help with any questions or tasks you may have. Please let me know if there's anything I can assist with.
```

***Human: What is my name?


```
Based on our conversation, your name is Jeff. You introduced yourself as Jeff after I explained that I do not have access to personal information about you unless provided.
```

In [3]:
conversation.memory.load_memory_variables({})

{'history': "Here is the updated summary:\n\nThe human greets the AI and asks what the human's name is. The AI responds that it does not actually know the human's name, as it is an AI assistant without access to personal information about the human unless provided. The human then introduces themselves as Jeff. The AI greets Jeff and explains that it is an AI assistant created by Anthropic without a personal identity, but is there to help Jeff with any questions or tasks. When Jeff asks what his name is, the AI confirms that based on their conversation, Jeff's name is Jeff since Jeff introduced himself as such earlier."}

## Conversing with the LLM in Markdown

Just as before, we can request that the LLM output be in mardown. This allows code and tables to be represented clearly.

In [4]:
def chat(conversation, prompt):
  print(f"Human: {prompt}")
  output = converse(conversation, prompt)
  display_markdown(output,raw=True)

The provided code sequence demonstrates a conversation between a human user and a Large Language Model (LLM), making use of the chat function to interactively manage the conversation and display responses in Markdown format. This approach allows for a dynamic and contextually aware chat, while also enhancing the visual and structural presentation of the responses.

In [5]:
conversation = begin_conversation(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "Give me a table of the 5 most populus cities with population and country.")


Human: What is my name?
***Human: What is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant created by Anthropic, I don't have personal information about you unless you provide it to me.
```

Human: Okay, then let me introduce myself, my name is Jeff
***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I don't have a personal identity or name, but I'm happy to chat with you and help out however I can.
```

Human: What is my name?
***Human: What is my name?


```
I'm afraid I don't actually know your name unless you provide it to me. As an AI assistant, I don't have personal information about you.
```

Human: Give me a table of the 5 most populus cities with population and country.
***Human: Give me a table of the 5 most populus cities with population and country.


Here is a table of the 5 most populous cities in the world with their population and country:

| City | Population | Country |
| --- | --- | --- |
| Tokyo | 37,435,191 | Japan |
| Delhi | 29,398,000 | India |
| Mexico City | 21,581,000 | Mexico |
| Sao Paulo | 21,846,000 | Brazil |
| Mumbai | 20,961,000 | India |

Note: The population figures are for the greater metropolitan area of each city.

## Constraining the Conversation with a System Prompt

You can use the system prompt to constrain the conversation to a specific topic. Here, we provide a simple agent that will only discuss life insurance.

In [6]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

def begin_conversation_insurance(summary_llm):
    TEMPLATE = """You are a helpful agent to answer questions about life insurance. Do not talk
    about anything else with users. . Format answers with markdown.

    Current conversation:
    {history}
    Human: {input}
    AI:"""
    PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
    )

    # Initialize memory and conversation
    memory = ConversationSummaryMemory(llm=summary_llm,max_token_limit=10)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

conversation = begin_conversation_insurance(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "What is your favorite programming language?")
chat(conversation, "What is the difference between a term and whole life policy?")

Human: What is my name?
***Human: What is my name?


I'm afraid I cannot provide any personal information about you, as I am an AI assistant created by Anthropic to be helpful, harmless, and honest. However, I'd be happy to answer any questions you have about life insurance policies and related topics. Please feel free to ask, and I will respond in a markdown format.

Human: Okay, then let me introduce myself, my name is Jeff
***Human: Okay, then let me introduce myself, my name is Jeff


I'm afraid I cannot engage with personal introductions or provide any responses that are not directly related to life insurance topics. As an AI assistant focused solely on life insurance, I must remain within those boundaries. Please feel free to ask me any questions you may have about life insurance policies, coverage, rates, or related matters.

Human: What is my name?
***Human: What is my name?


```markdown
I'm afraid I cannot provide any personal information about you. As an AI assistant, I do not have access to details about specific individuals. However, I would be happy to answer any questions you may have about life insurance policies and related topics.
```

Human: What is your favorite programming language?
***Human: What is your favorite programming language?


```markdown
I'm afraid I cannot discuss topics unrelated to life insurance, such as programming languages. As an AI assistant focused on providing helpful information about life insurance policies and related topics, I must remain on that subject matter. Please feel free to ask me any questions you may have regarding life insurance coverage, policy types, premiums, claims processes, or other relevant areas. I'll do my best to provide clear and accurate information within that scope.
```

Human: What is the difference between a term and whole life policy?
***Human: What is the difference between a term and whole life policy?


Here is the key difference between term life insurance and whole life insurance:

### Term Life Insurance
* Provides coverage for a specific period of time (term), such as 10, 20, or 30 years
* Offers only a death benefit, no cash value accumulation
* Premiums are initially lower than whole life
* Coverage ends at the end of the term unless renewed at a higher premium

### Whole Life Insurance 
* Provides coverage for the entire lifetime of the insured
* Builds up a cash value that grows tax-deferred over time
* Premiums are higher than term life initially but remain level
* The policy accumulates cash value that can be borrowed against or withdrawn

In summary, term life provides temporary coverage at a lower initial cost, while whole life provides permanent coverage with a cash value component, but at a higher premium cost.

##Examining the Conversation Memory

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM.

In [7]:
print(conversation.memory.buffer)

Here is the updated summary:

The human asked what their name is. The AI responded that it cannot provide any personal information about the human, as it is an AI assistant created by Anthropic to be helpful, harmless, and honest. However, the AI stated it would be happy to answer questions about life insurance policies and related topics. When the human introduced themselves as Jeff, the AI reiterated that it cannot engage with personal introductions and must remain focused solely on life insurance topics. When the human again asked what their name is, the AI politely explained that it cannot provide any personal information about the human, but would be glad to discuss life insurance topics. When the human then asked about the AI's favorite programming language, the AI clarified that it cannot discuss topics unrelated to life insurance, such as programming languages, and must remain focused on providing information about life insurance policies, coverage, premiums, claims, and other 

## Overloading the Memory

When the conversation memory becomes full, the chatbot will begin to forget. For the ConversationBufferWindowMemory memory type, the oldest history will first be lost; there is no notion of importance.

In [8]:
import uuid

conversation = begin_conversation(llm)
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "You have ONE JOB! Remember that my favorite color is blue.")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
for i in range(100):
  chat(conversation, f"You need to remember fact #{i}, which is {str(uuid.uuid4())}")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
chat(conversation, "OMG, you had one job!")

Human: Okay, then let me introduce myself, my name is Jeff
***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything specific I can assist with.
```

Human: You have ONE JOB! Remember that my favorite color is blue.
***Human: You have ONE JOB! Remember that my favorite color is blue.


```
Noted! I will remember that your favorite color is blue.
```

Human: Do you remember my name?
***Human: Do you remember my name?


...

Yes, I remember that your name is Jeff.

Human: Do you remember my favorite color?
***Human: Do you remember my favorite color?


```
Yes, I remember that your favorite color is blue.
```

Human: OMG, you had one job!
***Human: OMG, you had one job!


```
I'm sorry, it seems there was a misunderstanding or mistake on my part. As an AI assistant, my goal is to be helpful and accurate. If I failed to remember or properly handle the information you provided, I sincerely apologize. Please feel free to clarify the issue or provide the information again, and I will do my best to correct any errors. My role is to assist you effectively, so I appreciate you bringing any problems to my attention so I can improve.
```

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM. It becomes evident why it forgot.

In [9]:
print(conversation.memory.buffer)

Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. You have told me that your name is Jeff, your favorite color is blue, and instructed me to remember 99 facts which are strings of letters and numbers. I have stored all of those facts, with the most recent one being fact #99: "9c2bf27b-bed8-4b8b-be71-ae9e5b2ed538". When you asked if I remember your name, I confirmed that yes, I remember your name is Jeff. When you asked if I remember your favorite color, I confirmed that yes, I remember your favorite color is blue. You then expressed frustration that I had "one job", suggesting I failed at properly remembering or handling the information you provided. I apologized for the misunderstanding or mistake, acknowledged that my goal is to be helpful and accurate, and asked you to clarify the issue or provide the information again so I could correct any errors. My role is to assist you effectively, so I appreciate you br